In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq

In [5]:
cambio = pd.read_excel('../datasets/tc_turistas.xlsx',
                       sheet_name=0)

turista = pd.read_excel('../datasets/tc_turistas.xlsx',
                        sheet_name=2)

In [10]:
cambio['DOLAR Oficial'].fillna(method='ffill', inplace=True)
cambio['DOLAR Blue'].fillna(method='ffill', inplace=True)

In [12]:
cambio

,FECHA,DOLAR Oficial,DOLAR Blue
0,2017-06-01,16.25,16.30
1,2017-06-02,16.20,16.32
2,2017-06-03,16.20,16.32
3,2017-06-04,16.20,16.32
4,2017-06-05,16.20,16.34
...,...,...,...
999,2020-02-25,63.50,78.80
1000,2020-02-26,63.75,78.70
1001,2020-02-27,63.75,78.50
1002,2020-02-28,64.00,78.50
